In [14]:
import csv
import time
import requests
from bs4 import BeautifulSoup
from pattern.en import ngrams

In [15]:
Base_url = "http://www.moneycontrol.com"

# Build a dictionary of companies and their abbreviated names 
# companies = {'cadilahealthcare':'CHC','piramalenterprises':'PH05'}
# companies = {'cadilahealthcare':'CHC' }
companies = {'cadilahealthcare':'CHC','piramalenterprises':'PH05',
             'glenmarkpharma':'GP08','glaxosmithklinepharmaceuticals':'GSK',
             'sunpharmaceuticalindustries':'SPI','lupinlaboratories':'LL',
             'cipla':'C','aurobindopharma':'AP',
             'drreddyslaboratories':'DRL','divislaboratories':'DL03'}
             
# Create a list of the news section urls of the respective companies 
url_list = ['http://www.moneycontrol.com/company-article/{}/news/{}#{}'.format(k,v,v) for k,v in companies.items()]

In [16]:
def changeWord(word):
    for letter in word:
        if letter == "_":
            word = word.replace(letter," ")
    return word

In [17]:
url_list

['http://www.moneycontrol.com/company-article/cipla/news/C#C',
 'http://www.moneycontrol.com/company-article/drreddyslaboratories/news/DRL#DRL',
 'http://www.moneycontrol.com/company-article/aurobindopharma/news/AP#AP',
 'http://www.moneycontrol.com/company-article/sunpharmaceuticalindustries/news/SPI#SPI',
 'http://www.moneycontrol.com/company-article/piramalenterprises/news/PH05#PH05',
 'http://www.moneycontrol.com/company-article/divislaboratories/news/DL03#DL03',
 'http://www.moneycontrol.com/company-article/lupinlaboratories/news/LL#LL',
 'http://www.moneycontrol.com/company-article/cadilahealthcare/news/CHC#CHC',
 'http://www.moneycontrol.com/company-article/glenmarkpharma/news/GP08#GP08',
 'http://www.moneycontrol.com/company-article/glaxosmithklinepharmaceuticals/news/GSK#GSK']

In [18]:
# Create an empty list which will contain the selected news articles 
List_of_links = []   

# Extract the relevant news articles weblinks from the news section of selected companies
for urls in url_list:
    html = requests.get(urls)
    # Create a BeautifulSoup object
    soup = BeautifulSoup(html.text,'html.parser')  

    # Retrieve a list of all the links and the titles for the respective links
    word1,word2,word3 = "US","USA","USFDA"
 
    sub_links = soup.find_all('a', class_='arial11_summ')
    for links in sub_links:
        sp = BeautifulSoup(str(links),'html.parser')  # first convert into a string
        tag = sp.a
        if word1 in tag['title'] or word2 in tag['title'] or word3 in tag['title']:
            category_links = Base_url + tag["href"]
            List_of_links.append(category_links)

In [19]:
# Remove the duplicate news articles based on News Title
unique_links = list(set(List_of_links))

# Create a dictionary of positive/negative words related to the Pharma Sector
    
reader = csv.reader(open('dict1.csv', "rt"))
pharma_dict = dict((rows[0],rows[1]) for rows in reader)

# Creating an empty list which will be filled later with news article links, and Polarity values (pos/neg)
df =[]

In [20]:
for key, value in pharma_dict.items():
    key1 = changeWord(key)
    pharma_dict[key1] = pharma_dict.pop(key)

In [26]:
# Open the choosen news articles and extract the main text  
new_dict = {}
for selected_links in unique_links:
    results_url = selected_links 
    #print results_url
    try:
        results = requests.get(results_url)
        results_text = BeautifulSoup(results.text,"lxml")
        extract_text = results_text.find(class_='arti-flow')
        final_text = extract_text.get_text()
        
        # Pre-processing the extracted text using ngrams function from the pattern package   
        final_text1 = ngrams(final_text, n=1, punctuation=".,;:!?()[]{}`''\"@#$^&*+-|=~_", continuous=False)
        
        # Checking if any of the words in the news article text matches with the words in the Pharma dictionary(pos/neg)
        
        positive_score,negative_score = 0,0
   
        for x in final_text1:
            if x[0] in pharma_dict:
                new_dict[x[0]] = pharma_dict[x[0]] 
                  
        positive_list = [] ; negative_list = [];
        for key, value in new_dict.items():
            if value == 'positive': positive_list.append(key)
            if value == 'negative': negative_list.append(key)
                           
        # Compute the positive score, the negative score for each news articles
        positive_score = len(positive_list) ; negative_score = len(negative_list);
        #print positive_list ; print negative_list ;
        #print positive_score ; print negative_score;
        var_nos = [results_url,positive_score, negative_score]
        df.append(var_nos)  
    except:
        a = 1

In [27]:
# Print the final list of the Sentiment Analysis 
for item in df:
    print(item)

[u'http://www.moneycontrol.com/news/buzzing-stocks/cadila-healthcare4usfda-approval-for-itraconazole-capsules_9725221.html', 1, 0]
[u'http://www.moneycontrol.com/news/cnbc-tv18-comments/us-fda-issues-2-observations-to-aurobindo39s-unit-vihyderabad_9678621.html', 0, 0]
[u'http://www.moneycontrol.com/news/buzzing-stocks/sun-pharma-gains-4eirusfda-for-dadra-facility_9854941.html', 2, 1]
[u'http://www.moneycontrol.com/news/announcements/zydus-receives-final-approvalusfda-for-desvenlafaxine-extended-release-tablets_9856521.html', 2, 1]
[u'http://www.moneycontrol.com/news/business/pharma-stocks-extend-gains-divisdr-reddy39s-get-reprieveus-fda_9739321.html', 2, 3]
[u'http://www.moneycontrol.com/news/buzzing-stocks/dr-reddy39s-lab-rises-4launchkidney-drugus-market_9800301.html', 2, 4]
[u'http://www.moneycontrol.com/news/buzzing-stocks/cadila-healthcare-gains-2usfda-nod-for-desvenlafaxine-tablets_9856761.html', 2, 4]
[u'http://www.moneycontrol.com/news/buzzing-stocks/upbeat-clsa-view-procedural